# 📊 Descarga de datos de Generación Renovable - API REE

Este notebook descarga datos de generación eléctrica renovable/no renovable desde la **API de Red Eléctrica de España (REE)**.

## Especificaciones:
- **API Base**: `https://apidatos.ree.es`
- **Widget**: `/es/datos/generacion/evolucion-renovable-no-renovable`
- **Series**: 18 (España + 17 regiones)
- **Periodo**: Enero 2016 - Diciembre 2025
- **Estrategia**: Descarga año por año para evitar errores 500

## Mapeo de geo_ids corregido:
- 8741: Península (España)
- 8742: Canarias
- 8743: Baleares
- 4-21: CCAA peninsulares

---
## 1. Configuración e Instalación

In [23]:
# Instalar dependencias si no están disponibles
# !pip install requests pandas openpyxl tqdm

In [24]:
import os
import time
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import requests
import pandas as pd
from tqdm.notebook import tqdm

# Configurar pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Librerías cargadas correctamente")

✅ Librerías cargadas correctamente


---
## 2. Configuración de parámetros

In [25]:
# ============================================
# CONFIGURACIÓN PRINCIPAL
# ============================================

# Directorio de salida
OUTPUT_DIR = Path(r"G:\Mi unidad\Proyectos\IPA27_project\data\raw\renovables")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# API Base URL
API_BASE = "https://apidatos.ree.es"
WIDGET = "/es/datos/generacion/evolucion-renovable-no-renovable"

# Headers para la API
HEADERS = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Referer": "https://www.ree.es/",
    "Origin": "https://www.ree.es"
}

# ============================================
# MAPEO CORRECTO DE GEO_IDS
# ============================================
# Basado en la información proporcionada:
# - España peninsular: 8741
# - Canarias: 8742  
# - Baleares: 8743
# - CCAA peninsulares: 4-21 (sin 12, 19)

CCAA_MAP = {
    # Sistemas eléctricos principales
    8741: "España_Peninsula",
    8742: "Canarias",
    8743: "Baleares",
    # CCAA peninsulares
    4: "Andalucía",
    5: "Aragón",
    6: "Cantabria",
    7: "Castilla-La Mancha",
    8: "Castilla y León",
    9: "Cataluña",
    10: "País Vasco",
    11: "Principado de Asturias",
    13: "Comunidad de Madrid",
    14: "Comunidad Foral de Navarra",
    15: "Comunitat Valenciana",
    16: "Extremadura",
    17: "Galicia",
    20: "La Rioja",
    21: "Región de Murcia",
}

# Años a descargar (uno por uno para evitar error 500)
YEARS = list(range(2016, 2026))  # 2016 a 2025

print(f"📁 Directorio de salida: {OUTPUT_DIR}")
print(f"📊 Series a descargar: {len(CCAA_MAP)}")
print(f"📅 Años: {YEARS[0]} - {YEARS[-1]}")
print(f"📥 Total peticiones: {len(CCAA_MAP) * len(YEARS)}")

📁 Directorio de salida: G:\Mi unidad\Proyectos\IPA27_project\data\raw\renovables
📊 Series a descargar: 18
📅 Años: 2016 - 2025
📥 Total peticiones: 180


---
## 3. Funciones de descarga

In [26]:
def construir_url_params(geo_id: int, year: int) -> dict:
    """
    Construye los parámetros de la URL según el tipo de geo_id.
    
    - geo_ids 8741-8743: sistemas eléctricos (solo geo_limit, sin geo_trunc)
    - geo_ids 4-21: CCAA (usan geo_limit=ccaa + geo_trunc + geo_ids)
    """
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    
    params = {
        "start_date": start_date,
        "end_date": end_date,
        "time_trunc": "month",
    }
    
    # Para sistemas eléctricos (8741, 8742, 8743), usar geo_limit SIN geo_trunc
    if geo_id >= 8740:
        if geo_id == 8741:
            params["geo_limit"] = "peninsular"
        elif geo_id == 8742:
            params["geo_limit"] = "canarias"
        elif geo_id == 8743:
            params["geo_limit"] = "baleares"
        # NO incluir geo_trunc ni geo_ids para sistemas eléctricos
    else:
        # CCAA: incluir geo_trunc + geo_limit + geo_ids
        params["geo_trunc"] = "electric_system"
        params["geo_limit"] = "ccaa"
        params["geo_ids"] = geo_id
    
    return params


def descargar_datos_api(
    geo_id: int,
    year: int,
    max_retries: int = 3
) -> Optional[dict]:
    """
    Descarga datos de la API de REE con reintentos.
    """
    url = f"{API_BASE}{WIDGET}"
    params = construir_url_params(geo_id, year)
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, headers=HEADERS, timeout=60)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code in [500, 502, 503]:
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)
                    continue
            else:
                print(f"      ⚠️ HTTP {response.status_code}")
                return None
                
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            print(f"      ⚠️ Error: {e}")
            return None
    
    return None


def json_a_dataframe(data: dict, geo_id: int, ccaa_nombre: str) -> pd.DataFrame:
    """
    Convierte el JSON de la API a DataFrame.
    """
    registros = []
    
    try:
        included = data.get("included", [])
        
        for item in included:
            attrs = item.get("attributes", {})
            tipo = attrs.get("title", "")
            valores = attrs.get("values", [])
            
            for valor in valores:
                registros.append({
                    "geo_id": geo_id,
                    "region": ccaa_nombre,
                    "tipo": tipo,
                    "fecha": valor.get("datetime", ""),
                    "valor_MWh": valor.get("value", 0),
                    "porcentaje": valor.get("percentage", 0),
                })
    except Exception as e:
        print(f"      ⚠️ Error parseando JSON: {e}")
    
    df = pd.DataFrame(registros)
    
    if not df.empty:
        df["fecha"] = pd.to_datetime(df["fecha"])
    
    return df


print("✅ Funciones de descarga definidas")

✅ Funciones de descarga definidas


---
## 4. Probar descarga con una región

In [27]:
# Probar con España peninsular (geo_id=8741)
print("🧪 Probando España Peninsular...")
data = descargar_datos_api(8741, 2024)
if data:
    df_test = json_a_dataframe(data, 8741, "España_Peninsula")
    print(f"✅ Éxito! {len(df_test)} registros")
    display(df_test.head())
else:
    print("❌ Error")

🧪 Probando España Peninsular...
✅ Éxito! 24 registros


,geo_id,region,tipo,fecha,valor_MWh,porcentaje
0,8741,España_Peninsula,Renovable,2024-01-01 00:00:00+01:00,1.207199e+07,0.521983
1,8741,España_Peninsula,Renovable,2024-02-01 00:00:00+01:00,1.310837e+07,0.601819
2,8741,España_Peninsula,Renovable,2024-03-01 00:00:00+01:00,1.447302e+07,0.656903
3,8741,España_Peninsula,Renovable,2024-04-01 00:00:00+02:00,1.359943e+07,0.656176
4,8741,España_Peninsula,Renovable,2024-05-01 00:00:00+02:00,1.333417e+07,0.638945


---
## 5. 🚀 Ejecutar descarga completa

In [28]:
# ============================================
# DESCARGA DE TODAS LAS SERIES
# ============================================

resultados = []
datos_por_region = {}  # Almacenar DataFrames por región

total_peticiones = len(CCAA_MAP) * len(YEARS)
print(f"🚀 Iniciando descarga de {total_peticiones} peticiones...\n")

with tqdm(total=total_peticiones, desc="Progreso total") as pbar:
    for geo_id, region_nombre in CCAA_MAP.items():
        print(f"\n📍 {region_nombre} (geo_id={geo_id})")
        
        dfs_region = []
        exitos = 0
        errores = 0
        
        for year in YEARS:
            data = descargar_datos_api(geo_id, year)
            
            if data:
                df = json_a_dataframe(data, geo_id, region_nombre)
                if not df.empty:
                    dfs_region.append(df)
                    exitos += 1
                else:
                    errores += 1
            else:
                errores += 1
            
            pbar.update(1)
            time.sleep(0.3)  # Pausa entre peticiones
        
        # Combinar todos los años de esta región
        if dfs_region:
            df_region = pd.concat(dfs_region, ignore_index=True)
            df_region = df_region.drop_duplicates().sort_values("fecha")
            datos_por_region[region_nombre] = df_region
            print(f"   ✅ {exitos} años OK, {errores} errores, {len(df_region)} registros total")
        else:
            print(f"   ❌ Sin datos")
        
        resultados.append({
            "geo_id": geo_id,
            "region": region_nombre,
            "exitos": exitos,
            "errores": errores,
            "registros": len(df_region) if dfs_region else 0
        })

print("\n" + "="*60)
print("✅ DESCARGA COMPLETADA")
print("="*60)

🚀 Iniciando descarga de 180 peticiones...



Progreso total:   0%|          | 0/180 [00:00<?, ?it/s]


📍 España_Peninsula (geo_id=8741)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Canarias (geo_id=8742)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Baleares (geo_id=8743)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Andalucía (geo_id=4)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Aragón (geo_id=5)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Cantabria (geo_id=6)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Castilla-La Mancha (geo_id=7)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Castilla y León (geo_id=8)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Cataluña (geo_id=9)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 País Vasco (geo_id=10)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Principado de Asturias (geo_id=11)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Comunidad de Madrid (geo_id=13)
   ✅ 10 años OK, 0 errores, 240 registros total

📍 Comunidad Foral de Navarra (geo_id=14)
   ✅ 10 años OK, 0 errores, 240 registros 

---
## 6. Resumen de resultados

In [29]:
df_resultados = pd.DataFrame(resultados)

print("\n📊 RESUMEN DE DESCARGAS")
print("="*60)
display(df_resultados)

print(f"\nTotal regiones con datos: {len(datos_por_region)}")
print(f"Total registros: {sum(df_resultados['registros'])}")


📊 RESUMEN DE DESCARGAS


,geo_id,region,exitos,errores,registros
0,8741,España_Peninsula,10,0,240
1,8742,Canarias,10,0,240
2,8743,Baleares,10,0,240
3,4,Andalucía,10,0,240
4,5,Aragón,10,0,240
5,6,Cantabria,10,0,240
6,7,Castilla-La Mancha,10,0,240
7,8,Castilla y León,10,0,240
8,9,Cataluña,10,0,240
9,10,País Vasco,10,0,240



Total regiones con datos: 18
Total registros: 4320


---
## 7. Guardar archivos individuales por región

In [40]:
# ===== SECCIÓN 7: GUARDAR ARCHIVOS INDIVIDUALES =====
print("💾 Guardando archivos individuales...\n")

for region_nombre, df in datos_por_region.items():
    nombre_archivo = region_nombre.replace(" ", "_").replace(".", "").replace("/", "_")
    archivo = OUTPUT_DIR / f"{nombre_archivo}.xlsx"
    
    df_copy = df.copy()
    
    # IMPORTANTE: Convertir fecha a string (sin timezone)
    df_copy["fecha"] = df_copy["fecha"].astype(str).str[:10]
    
    df_pivot = df_copy.pivot_table(
        index="fecha",
        columns="tipo",
        values=["valor_MWh", "porcentaje"],
        aggfunc="first"
    )
    
    df_pivot.columns = [f"{col[1]}_{col[0]}" for col in df_pivot.columns]
    df_pivot = df_pivot.reset_index()
    
    df_pivot.to_excel(archivo, index=False, sheet_name=region_nombre[:30])
    print(f"   📄 {archivo.name}: {len(df_pivot)} filas")

print(f"\n✅ Archivos guardados en: {OUTPUT_DIR}")

💾 Guardando archivos individuales...

   📄 España_Peninsula.xlsx: 120 filas
   📄 Canarias.xlsx: 120 filas
   📄 Baleares.xlsx: 120 filas
   📄 Andalucía.xlsx: 120 filas
   📄 Aragón.xlsx: 120 filas
   📄 Cantabria.xlsx: 120 filas
   📄 Castilla-La_Mancha.xlsx: 120 filas
   📄 Castilla_y_León.xlsx: 120 filas
   📄 Cataluña.xlsx: 120 filas
   📄 País_Vasco.xlsx: 120 filas
   📄 Principado_de_Asturias.xlsx: 120 filas
   📄 Comunidad_de_Madrid.xlsx: 120 filas
   📄 Comunidad_Foral_de_Navarra.xlsx: 120 filas
   📄 Comunitat_Valenciana.xlsx: 120 filas
   📄 Extremadura.xlsx: 120 filas
   📄 Galicia.xlsx: 120 filas
   📄 La_Rioja.xlsx: 120 filas
   📄 Región_de_Murcia.xlsx: 120 filas

✅ Archivos guardados en: G:\Mi unidad\Proyectos\IPA27_project\data\raw\renovables


---
## 8. Compilar en archivo único con 18 hojas

In [41]:
# ===== SECCIÓN 8: COMPILAR ARCHIVO ÚNICO =====
archivo_compilado = OUTPUT_DIR / "renovables_compilado.xlsx"
print(f"📚 Compilando archivo único: {archivo_compilado.name}\n")

with pd.ExcelWriter(archivo_compilado, engine='openpyxl') as writer:
    for region_nombre, df in datos_por_region.items():
        sheet_name = region_nombre[:31].replace("/", "_")
        
        df_copy = df.copy()
        
        # IMPORTANTE: Convertir fecha a string (sin timezone)
        df_copy["fecha"] = df_copy["fecha"].astype(str).str[:10]
        
        df_pivot = df_copy.pivot_table(
            index="fecha",
            columns="tipo",
            values=["valor_MWh", "porcentaje"],
            aggfunc="first"
        )
        df_pivot.columns = [f"{col[1]}_{col[0]}" for col in df_pivot.columns]
        df_pivot = df_pivot.reset_index()
        
        df_pivot.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"   📋 {sheet_name}: {len(df_pivot)} filas")

print(f"\n✅ Archivo compilado: {archivo_compilado}")

📚 Compilando archivo único: renovables_compilado.xlsx

   📋 España_Peninsula: 120 filas
   📋 Canarias: 120 filas
   📋 Baleares: 120 filas
   📋 Andalucía: 120 filas
   📋 Aragón: 120 filas
   📋 Cantabria: 120 filas
   📋 Castilla-La Mancha: 120 filas
   📋 Castilla y León: 120 filas
   📋 Cataluña: 120 filas
   📋 País Vasco: 120 filas
   📋 Principado de Asturias: 120 filas


Exception ignored in: <function ZipFile.__del__ at 0x0000024AE15224C0>
Traceback (most recent call last):
  File "c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\zipfile.py", line 1807, in __del__
    self.close()
  File "c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\zipfile.py", line 1824, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x0000024AE15224C0>
Traceback (most recent call last):
  File "c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\zipfile.py", line 1807, in __del__
    self.close()
  File "c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\zipfile.py", line 1824, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


   📋 Comunidad de Madrid: 120 filas
   📋 Comunidad Foral de Navarra: 120 filas
   📋 Comunitat Valenciana: 120 filas
   📋 Extremadura: 120 filas
   📋 Galicia: 120 filas
   📋 La Rioja: 120 filas
   📋 Región de Murcia: 120 filas

✅ Archivo compilado: G:\Mi unidad\Proyectos\IPA27_project\data\raw\renovables\renovables_compilado.xlsx


---
## 9. Verificación final

In [42]:
# Listar archivos generados
print("\n📁 Archivos generados:")
for f in sorted(OUTPUT_DIR.glob("*.xlsx")):
    size_kb = f.stat().st_size / 1024
    print(f"   📄 {f.name} ({size_kb:.1f} KB)")


📁 Archivos generados:
   📄 Andalucía.xlsx (12.3 KB)
   📄 Aragón.xlsx (12.2 KB)
   📄 Baleares.xlsx (12.4 KB)
   📄 Canarias.xlsx (12.4 KB)
   📄 Cantabria.xlsx (12.1 KB)
   📄 Castilla-La_Mancha.xlsx (12.3 KB)
   📄 Castilla_y_León.xlsx (12.2 KB)
   📄 Cataluña.xlsx (12.3 KB)
   📄 Comunidad_de_Madrid.xlsx (12.1 KB)
   📄 Comunidad_Foral_de_Navarra.xlsx (12.2 KB)
   📄 Comunitat_Valenciana.xlsx (12.2 KB)
   📄 España_Peninsula.xlsx (12.5 KB)
   📄 Extremadura.xlsx (12.2 KB)
   📄 Galicia.xlsx (12.3 KB)
   📄 La_Rioja.xlsx (12.1 KB)
   📄 País_Vasco.xlsx (12.2 KB)
   📄 Principado_de_Asturias.xlsx (12.2 KB)
   📄 Región_de_Murcia.xlsx (12.2 KB)
   📄 renovables_compilado.xlsx (132.8 KB)


In [43]:
# Vista previa de una región
print("\n📈 Vista previa de datos:")
if datos_por_region:
    primera_region = list(datos_por_region.keys())[0]
    print(f"\n{primera_region}:")
    display(datos_por_region[primera_region].head(10))


📈 Vista previa de datos:

España_Peninsula:


,geo_id,region,tipo,fecha,valor_MWh,porcentaje
0,8741,España_Peninsula,Renovable,2016-01-01 00:00:00+01:00,9.828544e+06,0.442170
12,8741,España_Peninsula,No renovable,2016-01-01 00:00:00+01:00,1.239942e+07,0.557830
1,8741,España_Peninsula,Renovable,2016-02-01 00:00:00+01:00,1.109233e+07,0.522433
13,8741,España_Peninsula,No renovable,2016-02-01 00:00:00+01:00,1.013972e+07,0.477567
2,8741,España_Peninsula,Renovable,2016-03-01 00:00:00+01:00,1.123134e+07,0.494904
14,8741,España_Peninsula,No renovable,2016-03-01 00:00:00+01:00,1.146261e+07,0.505096
3,8741,España_Peninsula,Renovable,2016-04-01 00:00:00+02:00,1.106652e+07,0.519605
15,8741,España_Peninsula,No renovable,2016-04-01 00:00:00+02:00,1.023144e+07,0.480395
4,8741,España_Peninsula,Renovable,2016-05-01 00:00:00+02:00,1.065738e+07,0.525241
16,8741,España_Peninsula,No renovable,2016-05-01 00:00:00+02:00,9.633085e+06,0.474759


---
## 10. Guardar log de resultados

In [44]:
# Guardar log
log_path = OUTPUT_DIR / "_log_descargas.csv"
df_resultados.to_csv(log_path, index=False, encoding='utf-8-sig')
print(f"📝 Log guardado: {log_path}")

📝 Log guardado: G:\Mi unidad\Proyectos\IPA27_project\data\raw\renovables\_log_descargas.csv
